#### Importing classes and definitions from Structures
> To use any def or class from Structures precede it with 'S.'
> 
> Simply copy code instead is also an option

In [166]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [167]:
import ipynb.fs.defs.Structures as S

In [ ]:
# after the sweep we got a list containing intersection points
# main list - in book called 'D'    - better name needed I guess

D = intersections( set_of_subdivisions )


In [168]:
#test needed
# polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (2,1)], [(2,1), (1,1)]]
# polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(0,0)]]
polygon3 = [[(-5,-5), (-5,0)], [(-5,0), (0,5)], [(0,5), (1,0)], [(1,0), (0,-5)], [(0,-5), (-5,-5)]]
polygon1 = [[(1,1), (1,2)], [(1,2),(2,2)], [(2,2), (1,1)]]
polygon2 = [[(0,0), (0,5)], [(0,5), (5,5)], [(5,5),(0,0)]]
subdivision = [polygon1, polygon3, polygon2]
# subdivision = [polygon1]
# subdivision = [polygon3]


D = S.convert(subdivision)

UnboundLocalError: local variable 'i' referenced before assignment

> Operating on indexes is tiring but works - better ideas appreciated
> 
> For some reason following code does not work for triangles (???)

### Determining boundary cycles

In [159]:
import numpy as np
# kto by pomyślał że ten skurwiały wyznacznik się znowu przyda :))))
# det > 0 <=> c to the left of [a,b]
def det(a,b,c):                                            
    m = [a[0], a[1], b[0], b[1], c[0], c[1]]
    return np.linalg.det(np.concatenate((np.reshape(m, (3, 2)), np.ones((3,1))), axis=1))
    
class Boundary:
    
    def __init__(self, first_edge_index, boundary_index):
        # (technical) boundary index - perhaps redundant
        self.boundary_index = boundary_index
        
        # list of half_edges indexes in the_list forming the boundary
        self.edges = [first_edge_index]
        
        # boundary can be either outer or inner
        self.type = None
        
        # index of edge containing leftmost vertex as origin - easiest way to decide the type after boundary creation
        self.leftmost_edge = first_edge_index
        
        # leftmost vertex - perhabs redundant
        self.leftmost_vertex = None
     
    # adding new edge to boundary
    def add_edge(self, edge_index):
        # adding edge to edge list
        self.edges.append(edge_index)
        
        # checking whether added edge should replace leftmost edge (by origins)
        if D.v_list[ D.he_list[edge_index].origin ].coord[0] < D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[0]:
            self.leftmost_edge = edge_index
        #if tied in x-coordinates, choose one with smaller y-coordinate
        elif D.v_list[ D.he_list[edge_index].origin ].coord[0] == D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[0]:
            if D.v_list[ D.he_list[edge_index].origin ].coord[1] < D.v_list[ D.he_list[self.leftmost_edge].origin ].coord[1]:
                self.leftmost_edge = edge_index
    
    # deciding if boundary is inner/outer - method called after closing the boundary
    def set_type(self):
        if det (D.v_list[ D.he_list[ D.he_list[self.leftmost_edge].prev ].origin ].coord, 
                D.v_list[ D.he_list[ D.he_list[self.leftmost_edge].next ].origin ].coord, 
                D.v_list[  D.he_list[self.leftmost_edge].origin ].coord) > 0:
            self.type = "Inner"
        else:
            self.type = "Outer"
        
        self.leftmost_vertex = D.he_list[self.leftmost_edge].origin
        
        
        
# function returning list of boundaries from D
def get_boundaries( D ):
    
    boundaries_list = []
    boundary_index = 0
    
    #index of next half_edge added to some boundary
    processed_he = 0
    new_boundary_flag = True
    
    # this part here shits linear complexity - better solution is a bit complex, to be worried after all
    # list and method neccessary to choose next half_edge processed after last edge in boundary
    visited_half_edges = [False for i in D.he_list]
    def next_after_last():
        for he_index in range(len(D.he_list)):
            if not visited_half_edges[he_index]:
                return he_index
    
    #again not optimal
    while visited_half_edges.count(False) > 0:
        
        # first edge of new boundary
        if new_boundary_flag:
            
            #adding boundary and first edge to boundary
            boundaries_list.append( Boundary(processed_he, boundary_index) )
            
            # marking as visited
            visited_half_edges[processed_he] = True
            
            # moving to next half_edge
            processed_he = D.he_list[processed_he].next
            
            new_boundary_flag = False
            
        # last edge of boundary
        elif not new_boundary_flag and D.he_list[processed_he].next == boundaries_list[boundary_index].edges[0]:
            
            #adding to boundary
            boundaries_list[boundary_index].add_edge(processed_he)
            
            # marking as visited
            visited_half_edges[processed_he] = True
            
            # deciding the type of boundary
            boundaries_list[boundary_index].set_type()
            
            # moving to any unvisited half_edge
            processed_he = next_after_last()
            
            #incrementing boundary index
            boundary_index+=1
            
            new_boundary_flag = True
            
        # neither first nor last edge in boundary
        elif not new_boundary_flag:
            
            #adding to boundary
            boundaries_list[boundary_index].add_edge(processed_he)
            
            # marking as visited
            visited_half_edges[processed_he] = True
            
            # moving to next half_edge
            processed_he = D.he_list[processed_he].next
 
        
    return boundaries_list

In [160]:
# test of boundary cycles creation

def print_boundaries( boundaries_list ):
    for b in boundaries_list:
        print ("Boundary", b.boundary_index)
        print ("Leftmost vertex", D.v_list[b.leftmost_vertex].coord)
        print( "Type", b.type)
        for j in b.edges:
            print ("edge",j)
        print('---')


boundaries = get_boundaries( D )
print(' ')
print_boundaries( boundaries )


 
Boundary 0
Leftmost vertex (1, 1)
Type Inner
edge 0
edge 2
edge 4
---
Boundary 1
Leftmost vertex (1, 1)
Type Outer
edge 1
edge 5
edge 3
---
Boundary 2
Leftmost vertex (-5, -5)
Type Inner
edge 6
edge 8
edge 10
edge 12
edge 14
---
Boundary 3
Leftmost vertex (-5, -5)
Type Outer
edge 7
edge 15
edge 13
edge 11
edge 9
---
Boundary 4
Leftmost vertex (0, 0)
Type Inner
edge 16
edge 18
edge 20
---
Boundary 5
Leftmost vertex (0, 0)
Type Outer
edge 17
edge 21
edge 19
---


### Constructing graph neccessary for face computing

### Face computing